In [1]:
# -*- coding: utf-8 -*-
import gzip
import copy
import numpy as np
from PIL import Image
import paddle.v2 as paddle

## ResNet 50 Model

In [2]:
DATA_DIM=224*224*3
DICT_DIM=10000
image = paddle.layer.data(name="image", type=paddle.data_type.dense_vector(DATA_DIM))
target = paddle.layer.data(name="target", type=paddle.data_type.integer_value_sequence(DICT_DIM))
label = paddle.layer.data(name="label", type=paddle.data_type.integer_value_sequence(DICT_DIM))
paddle.init(use_gpu=False, trainer_count=1)


In [3]:
def conv_bn_layer(input,
                  ch_out,
                  filter_size,
                  stride,
                  padding,
                  active_type=paddle.activation.Relu(),
                  ch_in=None):
    tmp = paddle.layer.img_conv(
        input=input,
        filter_size=filter_size,
        num_channels=ch_in,
        num_filters=ch_out,
        stride=stride,
        padding=padding,
        act=paddle.activation.Linear(),
        bias_attr=False)
    return paddle.layer.batch_norm(input=tmp, act=active_type)


def shortcut(input, ch_out, stride):
    if input.num_filters != ch_out:
        return conv_bn_layer(input, ch_out, 1, stride, 0,
                             paddle.activation.Linear())
    else:
        return input


def basicblock(input, ch_out, stride):
    short = shortcut(input, ch_out, stride)
    conv1 = conv_bn_layer(input, ch_out, 3, stride, 1)
    conv2 = conv_bn_layer(conv1, ch_out, 3, 1, 1, paddle.activation.Linear())
    return paddle.layer.addto(
        input=[short, conv2], act=paddle.activation.Relu())


def bottleneck(input, ch_out, stride):
    short = shortcut(input, ch_out * 4, stride)
    conv1 = conv_bn_layer(input, ch_out, 1, stride, 0)
    conv2 = conv_bn_layer(conv1, ch_out, 3, 1, 1)
    conv3 = conv_bn_layer(conv2, ch_out * 4, 1, 1, 0,
                          paddle.activation.Linear())
    return paddle.layer.addto(
        input=[short, conv3], act=paddle.activation.Relu())


def layer_warp(block_func, input, ch_out, count, stride):
    conv = block_func(input, ch_out, stride)
    for i in range(1, count):
        conv = block_func(conv, ch_out, 1)
    return conv


def resnet_imagenet(input, depth=50):
    cfg = {
        18: ([2, 2, 2, 1], basicblock),
        34: ([3, 4, 6, 3], basicblock),
        50: ([3, 4, 6, 3], bottleneck),
        101: ([3, 4, 23, 3], bottleneck),
        152: ([3, 8, 36, 3], bottleneck)
    }
    stages, block_func = cfg[depth]
    conv1 = conv_bn_layer(
        input, ch_in=3, ch_out=64, filter_size=7, stride=2, padding=3)
    pool1 = paddle.layer.img_pool(input=conv1, pool_size=3, stride=2)
    res1 = layer_warp(block_func, pool1, 64, stages[0], 1)
    res2 = layer_warp(block_func, res1, 128, stages[1], 2)
    res3 = layer_warp(block_func, res2, 256, stages[2], 2)
    res4 = layer_warp(block_func, res3, 512, stages[3], 2)
    pool2 = paddle.layer.img_pool(
        input=res4, pool_size=7, stride=1, pool_type=paddle.pooling.Avg())
    output = paddle.layer.fc(pool2, act=paddle.activation.Softmax(), size=1000)
    return pool2


In [4]:
def decoder(features, target, dict_dim, embed_size=512, label=None, is_train=True):
    
    encoded_features = paddle.layer.fc(
        input=features,
        size=512,
        act=paddle.activation.Relu(),
        name='encoded_features'
    )
        
    input_emb = paddle.layer.embedding(target, size=embed_size, name='embedding')
    input_emb = paddle.layer.fc(input=input_emb, size=512*3)
    
    grus = paddle.networks.gru_group(
        input=[input_emb],
        memory_boot=encoded_features,
        size=512,
        name='gru_group_layer'
    )
    # input_ = paddle.layer.concat(input=inputs)
    
#     rnn = paddle.networks.bidirectional_gru(input=inputs, size=1024, return_seq=True)
    
#     for i in xrange(num_layers-1):
#         rnn = paddle.networks.simple_lstm(input=rnn, size=512)
    
    output = paddle.layer.fc(input=grus, size=dict_dim, act=paddle.activation.Softmax())
    
    if is_train:
        cost = paddle.layer.classification_cost(input=output, label=label)
        return cost
    else:
        last_word = paddle.layer.last_seq(input=output)
        return last_word    

In [5]:
def train_caption_net(input_images, target, label, dict_dim):
    
    encoder_ = resnet_imagenet(input_images)
    cost = decoder(features=encoder_, target=target, label=label, dict_dim=dict_dim)
    
    return cost
    

In [6]:
def event_handler(event):
    if isinstance(event, paddle.event.EndIteration):
        if event.batch_id % 1 == 0:
            print "\nPass %d, Batch %d, Cost %f, %s" % (
                event.pass_id, event.batch_id, event.cost, event.metrics)
    if isinstance(event, paddle.event.EndPass):
        with gzip.open('params_pass_%d.tar.gz' % event.pass_id, 'w') as f:
            parameters.to_tar(f)

        result = trainer.test(reader=test_reader)
        print "\nTest with Pass %d, %s" % (event.pass_id, result.metrics)

In [7]:
DICT_DIM=10000

In [8]:
cost = train_caption_net(dict_dim=DICT_DIM, target=target, label=label, input_images=image)

[INFO 2018-01-31 14:21:49,397 layers.py:2689] output for __conv_0__: c = 64, h = 112, w = 112, size = 802816
[INFO 2018-01-31 14:21:49,402 layers.py:3251] output for __batch_norm_0__: c = 64, h = 112, w = 112, size = 802816
[INFO 2018-01-31 14:21:49,410 layers.py:2829] output for __pool_0__: c = 64, h = 56, w = 56, size = 200704
[INFO 2018-01-31 14:21:49,417 layers.py:2689] output for __conv_1__: c = 256, h = 56, w = 56, size = 802816
[INFO 2018-01-31 14:21:49,423 layers.py:3251] output for __batch_norm_1__: c = 256, h = 56, w = 56, size = 802816
[INFO 2018-01-31 14:21:49,432 layers.py:2689] output for __conv_2__: c = 64, h = 56, w = 56, size = 200704
[INFO 2018-01-31 14:21:49,439 layers.py:3251] output for __batch_norm_2__: c = 64, h = 56, w = 56, size = 200704
[INFO 2018-01-31 14:21:49,445 layers.py:2689] output for __conv_3__: c = 64, h = 56, w = 56, size = 200704
[INFO 2018-01-31 14:21:49,452 layers.py:3251] output for __batch_norm_3__: c = 64, h = 56, w = 56, size = 200704
[INFO 2

[INFO 2018-01-31 14:21:49,835 layers.py:3251] output for __batch_norm_36__: c = 1024, h = 14, w = 14, size = 200704
[INFO 2018-01-31 14:21:49,840 layers.py:2689] output for __conv_37__: c = 256, h = 14, w = 14, size = 50176
[INFO 2018-01-31 14:21:49,846 layers.py:3251] output for __batch_norm_37__: c = 256, h = 14, w = 14, size = 50176
[INFO 2018-01-31 14:21:49,850 layers.py:2689] output for __conv_38__: c = 256, h = 14, w = 14, size = 50176
[INFO 2018-01-31 14:21:49,855 layers.py:3251] output for __batch_norm_38__: c = 256, h = 14, w = 14, size = 50176
[INFO 2018-01-31 14:21:49,861 layers.py:2689] output for __conv_39__: c = 1024, h = 14, w = 14, size = 200704
[INFO 2018-01-31 14:21:49,865 layers.py:3251] output for __batch_norm_39__: c = 1024, h = 14, w = 14, size = 200704
[INFO 2018-01-31 14:21:49,870 layers.py:2689] output for __conv_40__: c = 256, h = 14, w = 14, size = 50176
[INFO 2018-01-31 14:21:49,878 layers.py:3251] output for __batch_norm_40__: c = 256, h = 14, w = 14, size 

In [9]:
adam_optimizer = paddle.optimizer.Adam(
    learning_rate=1e-4,
    regularization=paddle.optimizer.L2Regularization(rate=8e-4),
    model_average=paddle.optimizer.ModelAverage(average_window=0.5)
)

In [15]:
label = np.random.randint(2,10000, 11)
label[-1] = 1
target = np.zeros(11).astype('int')
target[1:11] = label[:-1]
target[0] = 0
print(target)
print(label)

[   0  917 3658 6736 3379 1297 1595 7641 8177 7995 7563]
[ 917 3658 6736 3379 1297 1595 7641 8177 7995 7563    1]


In [16]:
def create_reader():
    
    def reader():
        
        for i in xrange(10):
        
            image = np.random.randn(224*224*3) * 255
            label = np.random.randint(2,10000, 11)
            label[-1] = 1
            target = np.zeros(11).astype('int')
            target[1:11] = label[:-1]
            target[0] = 0

            yield image, target, label
    return reader
        

In [17]:
train_reader = create_reader()
train_batch = paddle.batch(reader=paddle.reader.shuffle(train_reader, buf_size=10),batch_size=4)

In [18]:
feeding = {'image':0, 'label':2, 'target':1}

In [19]:
parameters = paddle.parameters.create(cost)
parameters.init_from_tar(gzip.open('params/Paddle_ResNet50.tar.gz'))

In [20]:
trainer = paddle.trainer.SGD(cost=cost,parameters=parameters,update_equation=adam_optimizer)

In [21]:
trainer.train(reader=train_batch,event_handler=event_handler,feeding=feeding,num_passes=10)


Pass 0, Batch 0, Cost 101.391457, {'classification_error_evaluator': 1.0}

Pass 0, Batch 1, Cost 101.426231, {'classification_error_evaluator': 1.0}

Pass 0, Batch 2, Cost 101.229340, {'classification_error_evaluator': 1.0}


KeyboardInterrupt: 